In [4]:
import open_clip
import torch, torchvision

In [6]:
image = torchvision.io.read_image('/DQU-CIR/data/CIRR/dev/dev-0-0-img0.png')
print(type(image))
print(image)

<class 'torch.Tensor'>
tensor([[[156, 134, 139,  ..., 161, 170, 170],
         [136, 128, 140,  ..., 177, 166, 166],
         [129, 131, 137,  ..., 180, 176, 176],
         ...,
         [195, 200, 208,  ..., 197, 201, 201],
         [198, 196, 198,  ..., 200, 192, 192],
         [203, 195, 191,  ..., 203, 182, 182]],

        [[153, 131, 136,  ..., 150, 160, 160],
         [133, 125, 137,  ..., 166, 156, 156],
         [126, 128, 134,  ..., 171, 166, 166],
         ...,
         [195, 200, 208,  ..., 197, 201, 201],
         [198, 196, 198,  ..., 199, 191, 191],
         [203, 195, 191,  ..., 202, 181, 181]],

        [[148, 126, 131,  ..., 146, 158, 158],
         [128, 120, 132,  ..., 162, 154, 154],
         [121, 123, 129,  ..., 166, 164, 164],
         ...,
         [195, 200, 208,  ..., 195, 199, 199],
         [198, 196, 198,  ..., 197, 189, 189],
         [203, 195, 191,  ..., 200, 179, 179]]], dtype=torch.uint8)


In [7]:
from PIL import Image
image = Image.open('/DQU-CIR/data/CIRR/dev/dev-0-0-img0.png')
print(type(image))
print(image)

<class 'PIL.JpegImagePlugin.JpegImageFile'>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=570x627 at 0x7862F15CECA0>


In [2]:
category = 'shirt'
batch_size=16
lr=1e-5
patience = 5
model_name = 'hf-hub:laion/CLIP-ViT-B-16-laion2B-s34B-b88K'
pretrained = None
hidden_dim=512
dropout = 0.5

In [3]:
clip, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(model_name=model_name,pretrained=pretrained)
print(preprocess_train)
print(preprocess_val)

Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.9, 1.0), ratio=(0.75, 1.3333), interpolation=bicubic, antialias=True)
    <function _convert_to_rgb at 0x7862f26d19d0>
    ToTensor()
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
)
Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    <function _convert_to_rgb at 0x7862f26d19d0>
    ToTensor()
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
)


In [1]:
import torchvision.transforms as transforms
from PIL import Image

preprocess_train = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=transforms.InterpolationMode.BICUBIC, antialias=True),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=15),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                         std=[0.26862954, 0.26130258, 0.27577711])
])
print(preprocess_train)

Compose(
    Lambda()
    RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=bicubic, antialias=True)
    RandomHorizontalFlip(p=0.5)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.1, 0.1))
    RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1), scale=(0.9, 1.1), shear=[-10.0, 10.0])
    RandomApply(
    p=0.1
    GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))
)
    ToTensor()
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
)


In [20]:
import cv2
import numpy as np

def draw_text(img, point, text):
    fontScale = 0.5
    color = (0, 255, 0)  # Màu chữ (Xanh dương)
    thickness = 2
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    # In chữ lên ảnh
    cv2.putText(img, str(text), point, fontFace, fontScale, color, thickness, lineType=cv2.LINE_AA)
    return img


def draw_text_line(img, point, text_line: str):
    fontScale = 0.7
    # Tách các dòng chữ
    lines = text_line.split(", ")
    for i, text in enumerate(lines):
        if text:
            # Xác định vị trí của từng dòng
            draw_point = (point[0], point[1] + int(20 * i * fontScale))
            img = draw_text(img, draw_point, text)
    return img

# Ví dụ sử dụng (màu trắng):
img = cv2.imread("/DQU-CIR/data/FashionIQ/resized_image/shirt/978980539X.jpg")  # Thay bằng đường dẫn ảnh của bạn
text_to_draw = "maroon, sports graphic, logo"
point = (50, 50)
img = draw_text_line(img, point, text_to_draw)
cv2.imwrite("/DQU-CIR/B00A0AAD20.jpg", img)

True

In [ ]:
# preprocess_train = transforms.Compose([
#     transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
#     transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=transforms.InterpolationMode.BICUBIC, antialias=True),
#     # transforms.RandomHorizontalFlip(p=0.5),
#     # transforms.RandomVerticalFlip(p=0.5),
#     # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
#     transforms.RandomRotation(degrees=15),
#     transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],std=[0.26862954, 0.26130258, 0.27577711])
# ])

In [ ]:
# for image in list_image:
#     old = os.path.join(resized_image_category_path, f"{image} .jpg")
#     new = os.path.join(resized_image_category_path, f"{image}.jpg")
#     os.rename(old, new)

In [ ]:
# import matplotlib.pyplot as plt

# for batch in trainloader:
#     break
# print(batch['candidate_texts'][0])

# img = batch['candidate_imgs'][0].permute(1, 2, 0).cpu().numpy()  # Đổi thứ tự kênh từ (C, H, W) -> (H, W, C)
# # Hiển thị ảnh
# plt.figure(figsize=(6, 6))
# plt.imshow(img)
# plt.axis('off')
# plt.show()

# img = batch['target_imgs'][0].permute(1, 2, 0).cpu().numpy()  # Đổi thứ tự kênh từ (C, H, W) -> (H, W, C)
# # Hiển thị ảnh
# plt.figure(figsize=(6, 6))
# plt.imshow(img)
# plt.axis('off')
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# def display_original_image(image_tensor):
#     if len(image_tensor.shape) == 3:
#         num_channels = image_tensor.shape[0]
#         if num_channels == 3:  # Assuming RGB channels
#             mean = torch.tensor([0.48145466, 0.4578275, 0.40821073]).view(-1, 1, 1)
#             std = torch.tensor([0.26862954, 0.26130258, 0.27577711]).view(-1, 1, 1)
#             denormalized_image = image_tensor.mul(std) + mean
#         else:
#             raise NotImplementedError("Normalization reversal for non-RGB images not implemented yet")
#         denormalized_image = denormalized_image.permute(1, 2, 0).squeeze()  # Rearrange and remove batch dimension
#         pil_image = PIL.Image.fromarray(denormalized_image.mul(255).byte().numpy())
#         plt.figure(figsize=(6, 6))
#         plt.imshow(pil_image)
#         plt.axis('off')
#         plt.show()
#     else:
#         raise ValueError("Input image tensor must have 3 dimensions (C, H, W)")
# for batch in trainloader:
#     candidate_text = batch['candidate_texts'][0]
#     candidate_img = batch['candidate_imgs'][0]
#     target_img = batch['target_imgs'][0]
#     candidate = batch['candidates'][0]
#     target = batch['targets'][0]
#     print(f'/DQU-CIR/data/FashionIQ/resized_image/dress/{candidate}.jpg')
#     print(f'/DQU-CIR/data/FashionIQ/resized_image/dress/{target}.jpg')
#     display_original_image(candidate_img)
#     display_original_image(target_img)
#     break

In [2]:
import torch
x = torch.tensor([0, 1, 2, 3, 4, 5])
mask = torch.tensor([True, False, False, True, False, False])
print(x[mask])

tensor([0, 3])


In [7]:
a = torch.tensor([[10]])
a.item()

10

In [ ]:
from PIL import Image
def display_original_image(image_tensor):
    if len(image_tensor.shape) == 3:
        num_channels = image_tensor.shape[0]
        if num_channels == 3:  # Assuming RGB channels
            mean = torch.tensor([0.48145466, 0.4578275, 0.40821073]).view(-1, 1, 1)
            std = torch.tensor([0.26862954, 0.26130258, 0.27577711]).view(-1, 1, 1)
            denormalized_image = image_tensor.mul(std) + mean
        else:
            raise NotImplementedError("Normalization reversal for non-RGB images not implemented yet")
        denormalized_image = denormalized_image.permute(1, 2, 0).squeeze()  # Rearrange and remove batch dimension
        pil_image = Image.fromarray(denormalized_image.mul(255).byte().numpy())
        plt.figure(figsize=(6, 6))
        plt.imshow(pil_image)
        plt.axis('off')
        plt.show()
    else:
        raise ValueError("Input image tensor must have 3 dimensions (C, H, W)")

# for batch in devloader:
#     image_splits_dataset_path = os.path.join(image_splits_path, f"split.rc2.dev.json")
#     with open(image_splits_dataset_path) as f:
#         image_splits_dataset = json.load(f)
#     reference = batch['references'][0]
#     target_hard = batch['target_hards'][0]
#     reference_path = os.path.join(cirr_path, image_splits_dataset[reference].replace('./', ''))
#     target_hard_path = os.path.join(cirr_path, image_splits_dataset[target_hard].replace('./', ''))
#     print(reference_path)
#     print(target_hard_path)
#     print(batch['reference_texts'][0])
#     display_original_image(batch['reference_imgs'][0])
#     display_original_image(batch['target_hard_imgs'][0])
#     break

In [13]:
print("/DQU-CIR/data/FashionIQ/resized_image/dress/B00EV1B9C2.jpg") # target
print("/DQU-CIR/data/FashionIQ/resized_image/dress/B00BLY2L2E.jpg") #candidate

/DQU-CIR/data/FashionIQ/resized_image/dress/B00EV1B9C2.jpg
/DQU-CIR/data/FashionIQ/resized_image/dress/B00BLY2L2E.jpg


In [24]:
print("/DQU-CIR/data/CIRR/test1/test1-1001-2-img0.png")

/DQU-CIR/data/CIRR/test1/test1-1001-2-img0.png
